In [1]:
import pandas as pd
import ast
import gzip
import os

In [2]:
def descompimir_json(ruta, variable_anidada):
    # recibe una ruta de un archivo json anidado y carga la información en un DF
    fila = []
    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
      for line in archivo.readlines():
          fila.append(ast.literal_eval(line))

    df = pd.DataFrame(fila) ## leemos cada una de las filas

    ## desanidad el contenido de items y se agrega una nueva
    ## fila por cada elemento deitems

    ## Reseteamos los indices del dataframe
    df = df.explode(variable_anidada).reset_index()

    ## Eliminamos los indices pasados (repetidos)
    df = df.drop(columns="index")

    ## Concatenamos con user_id con todas las columans aplanadas del json.
    data_plana = pd.json_normalize(df[variable_anidada])[['item_id', 'playtime_forever','playtime_2weeks']]
    df = pd.concat([df['user_id'], data_plana ], axis=1)

    return df

In [3]:
path_data =  os.path.join('../data/original/users_items.json.gz')
user_items = descompimir_json(path_data,'items')

In [4]:
backup=user_items.copy()

In [ ]:
#user_items=backup.copy()

In [5]:
backup

,user_id,playtime_forever,item_id
0,76561197970982479,6.0,10
1,76561197970982479,0.0,20
2,76561197970982479,7.0,30
3,76561197970982479,0.0,40
4,76561197970982479,0.0,50
...,...,...,...
5170010,76561198329548331,0.0,373330
5170011,76561198329548331,3.0,388490
5170012,76561198329548331,4.0,521570
5170013,76561198329548331,3.0,519140


In [6]:
user_items

,user_id,playtime_forever,item_id
0,76561197970982479,6.0,10
1,76561197970982479,0.0,20
2,76561197970982479,7.0,30
3,76561197970982479,0.0,40
4,76561197970982479,0.0,50
...,...,...,...
5170010,76561198329548331,0.0,373330
5170011,76561198329548331,3.0,388490
5170012,76561198329548331,4.0,521570
5170013,76561198329548331,3.0,519140


In [7]:
user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   playtime_forever  float64
 2   item_id           object 
dtypes: float64(1), object(2)
memory usage: 118.3+ MB


In [8]:
user_items['item_id'] = user_items['item_id'].astype(float)

In [9]:
user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   playtime_forever  float64
 2   item_id           float64
dtypes: float64(2), object(1)
memory usage: 118.3+ MB


In [10]:
user_items.describe()

,playtime_forever,item_id
count,5.153209e+06,5.153209e+06
mean,9.914951e+02,1.784481e+05
std,5.418204e+03,1.318598e+05
min,0.000000e+00,1.000000e+01
25%,0.000000e+00,3.446000e+04
50%,3.400000e+01,2.144200e+05
75%,3.550000e+02,2.664300e+05
max,6.427730e+05,5.307200e+05


In [11]:
user_items.isna().sum()

user_id                 0
playtime_forever    16806
item_id             16806
dtype: int64